In [1]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [2]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain_openai import ChatOpenAI

In [3]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_tagging_chain(schema, llm)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.invoke(inp)

{'input': 'Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!',
 'text': {'sentiment': 'positive', 'language': 'Spanish'}}

In [5]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.invoke(inp)

{'input': 'Estoy muy enojado con vos! Te voy a dar tu merecido!',
 'text': {'sentiment': 'enojado', 'aggressiveness': 1, 'language': 'es'}}

# Finer control

Careful schema definition gives us more control over the model’s output.

Specifically, we can define:

- possible values for each property
- description to make sure that the model understands the property
- required properties to be returned
Here is an example of how we can use _enum_, _description_, and _required_ to control for each of the previously mentioned aspects:

In [6]:
schema = {
    "properties": {
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["language", "sentiment", "aggressiveness"],
}

In [7]:
chain = create_tagging_chain(schema, llm)

In [9]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.invoke(inp)

{'input': 'Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!',
 'text': {'aggressiveness': '3', 'language': 'spanish'}}

In [10]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.invoke(inp)

{'input': 'Estoy muy enojado con vos! Te voy a dar tu merecido!',
 'text': {'aggressiveness': '5', 'language': 'spanish'}}

In [11]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.invoke(inp)

{'input': 'Weather is ok here, I can go outside without much more than a coat',
 'text': {'aggressiveness': '3', 'language': 'english'}}

# Pydantic
We can also use a Pydantic schema to specify the required properties and types.

We can also send other arguments, such as enum or description, to each field.

This lets us specify our schema in the same manner that we would a new class or function in Python with purely Pythonic types.

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [13]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [14]:
chain = create_tagging_chain_pydantic(Tags, llm)

In [15]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.invoke(inp)

In [16]:
res

{'input': 'Estoy muy enojado con vos! Te voy a dar tu merecido!',
 'text': Tags(sentiment='sad', aggressiveness=5, language='spanish')}